In [34]:
import numpy as np
from ase import Atoms
from copy import deepcopy
from itertools import combinations
import pandas as pd
from ase.io import read, write
from configurations import build_lattice, create_configurations

In [35]:
from ase.visualize import view

In [36]:
from ase.build import molecule

In [37]:
cn = molecule('CN')
cn.positions

array([[ 0.      ,  0.      , -0.611046],
       [ 0.      ,  0.      ,  0.523753]])

In [38]:
cn.positions
cn.set_cell([10,10,10])
cn.pbc = True
cn.center()
cn.rotate(v='x',a=np.pi/2., center='COM')

In [39]:
view(cn)

In [40]:
lattice = build_lattice(4, motif=cn,lattice_constant=3)

In [41]:
lattice

[Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...)]

In [42]:
conf = Atoms()

In [43]:
for point in lattice:
    conf.extend(point)

In [44]:
view(conf)

In [47]:
confs = create_configurations(lattice,axis = 'z', angle = 180.0)

In [48]:
confs

{'AAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...)}

In [50]:
view(confs.values())

In [76]:
bmim_pf6_opt = read('./opt_coords_bigbox.xyz')
bmim_pf6_opt.cell = [44.20800018, 42.54000092, 50.]
bmim_pf6_opt.center()

In [77]:
view(bmim_pf6_opt)

In [78]:
pf6 = bmim_pf6_opt[0:7]; bmim = bmim_pf6_opt[7:32]; electrode = bmim_pf6_opt[32:]

In [79]:
pair = pf6+bmim
eq_dist_ion_pair_electrode = abs(electrode.get_center_of_mass()-pair.get_center_of_mass())[2]
eq_dist_ion_pair_electrode

4.4099961471344038

In [80]:
view(pair)

In [81]:
electrode.center()
pair.center()
pair.translate([0.0,0.0,-eq_dist_ion_pair_electrode])

In [62]:
view(pair)

In [63]:
lattice = build_lattice(4, motif=pair,lattice_constant=11.0)

In [65]:
confs = create_configurations(lattice,axis = 'z', angle=180.0)

In [66]:
confs

{'AAAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AAAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AACA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AACC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACCA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACCC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CAAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CAAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CACA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CACC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCCA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCCC': Atoms(symbols='C32H60F24N8P4', pbc=False)}

In [68]:
view(confs.values())

In [83]:
vector = 'z'
rotation_angle = 180.0
separation = 11.
pairs = 2
lshift = 0.0
rshift = 0.0

In [84]:
lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
l_confs = create_configurations(lattice,vector,rotation_angle)
######
for key in l_confs.keys():
    l_confs[key].set_cell(bmim_pf6_opt.cell)
    l_confs[key].center(axis=(0,1))
    if lshift != 0.0:
        l_confs[key].center(axis=2)
        l_confs[key].translate([0.0,0.0,lshift])
        if add_electrode and too_close(l_confs[key]+electrode,1.0):
            (l_confs[key]+electrode).write('lhs_'+key+'_positions.vasp')
            print('LHS ions are too close to the electrode')
            sys.exit()

lattice = build_lattice(pairs, motif=pair,lattice_constant=separation)
r_confs = create_configurations(lattice,vector,rotation_angle)
for key in r_confs.keys():
    r_confs[key].set_cell(bmim_pf6_opt.cell)
    r_confs[key].center(axis=(0,1))
    r_confs[key].translate([0.0,0.0,2*eq_dist_ion_pair_electrode])
    if rshift != 0.0:
        r_confs[key].center(axis=2)
        r_confs[key].translate([0.0,0.0,rshift])
        if add_electrode and too_close(r_confs[key]+electrode,1.0):
            (r_confs[key]+electrode).write('rhs_'+key+'_positions.vasp')
            print('RHS ions are too close to the electrode')
            sys.exit()

In [87]:
view(l_confs.values())

In [88]:
view(r_confs.values())

In [89]:
add_electrode = True

In [90]:
full_confs = []
for  l_key in l_confs.keys():
    for  r_key in r_confs.keys():
        box = l_confs[l_key] + r_confs[r_key]
        if add_electrode:
            box.extend(electrode)
        full_confs.append(box)

In [91]:
view(full_confs)

In [92]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)